<center><h1>Least-Squares Support Vector machine</h1></center>

## Summary:
1. [Introduction](#introduction)

2. [LSSVM CPU implementation](#lssvm_cpu)
    
3. [LSSVM GPU implementation](#lssvm_gpu)

4. [Discussing performance](#discussing_performance)

# 1. Introduction <a class="anchor" id="introduction"></a>

The Least-Squares Support Vector Machine (LSSVM) is a variation of the original Support Vector Machine (SVM) in which we have a slight change in the objective and restriction functions that results in a big simplification of the optimization problem.

First, let's see the optimization problem of an SVM:

$$ 
\begin{align}
    minimize && f_o(\vec{w},\vec{\xi})=\frac{1}{2} \vec{w}^T\vec{w} + C \sum_{i=1}^{n} \xi_i &&\\
    s.t. && d_i(\vec{w}^T\vec{x}_i+b)\geq 1 - \xi_i, && i = 1,..., n \\
         && \xi_i \geq 0,                            && i = 1,..., n
\end{align}
$$

In this case, we have a set of inequality restrictions and when solving the optimization problem by it's dual we find a discriminative function, adding the kernel trick, of the type:


$$ f(\vec{x}) = sign \ \Big( \sum_{i=1}^{n} \alpha_i^o d_i K(\vec{x}_i,\vec{x}) + b_o \Big) $$

Where $\alpha_i^o$ and $b_o$ denote optimum values. Giving enough regularization (smaller values of $C$) we get a lot of $\alpha_i^o$ nulls, resulting in a sparse model in which we only need to save the pairs $(\vec{x}_i,d_i)$ which have the optimum dual variable not null. The vectors $\vec{x}_i$ with not null $\alpha_i^o$ are known as support vectors (SV).



In the LSSVM case, we change the inequality restrictions to equality restrictions. As the $\xi_i$ may be negative we square its values in the objective function:

$$ 
\begin{align}
    minimize && f_o(\vec{w},\vec{\xi})=\frac{1}{2} \vec{w}^T\vec{w} + \gamma \frac{1}{2}\sum_{i=1}^{n} \xi_i^2 &&\\
    s.t. && d_i(\vec{w}^T\vec{x}_i+b) = 1 - \xi_i, && i = 1,..., n
\end{align}
$$


The dual of this optimization problem results in a system of linear equations, a set of Karush-Khun-Tucker (KKT) equations:

$$
\begin{bmatrix} 
    0 & \vec{d}^T \\
    \vec{d} & \Omega + \gamma^{-1} I 
\end{bmatrix}
\
\begin{bmatrix} 
    b  \\
    \vec{\alpha}
\end{bmatrix}
=
\begin{bmatrix} 
    0 \\
    \vec{1}
\end{bmatrix}
$$

Where, with the kernel trick, &nbsp; $\Omega_{i,j} = d_i d_j K(\vec{x}_i,\vec{x}_j)$,  &nbsp;  $\vec{d} = [d_1 \ d_2 \ ... \ d_n]^T$, &nbsp; $\vec{\alpha} = [\alpha_1 \ \alpha_2 \ ... \ \alpha_n]^T$ &nbsp;  e &nbsp; $\vec{1} = [1 \ 1 \ ... \ 1]^T$.

The discriminative function of the LSSVM has the same form of the SVM but the $\alpha_i^o$ aren't usually null, resulting in a bigger model. The big advantage of the LSSVM is in finding it's parameters, which is reduced to solving the linear system of the type:

$$ A\vec{x} = \vec{b} $$

A well-known solution of the linear system is when we minimize the square of the residues, that can be written as the optimization problem:

$$
\begin{align}
    minimize && f_o(\vec{x})=\frac{1}{2}||A\vec{x} - \vec{b}||^2\\
\end{align}
$$

And have the analytical solution:

$$ \vec{x} = A^{\dagger} \vec{b} $$

Where $A^{\dagger}$ is the pseudo-inverse defined as:

$$ A^{\dagger} = (A^T A)^{-1} A^T$$

In [1]:
%run -i 'load_dataset.py' # loading dataset
%run -i 'aux_func.py'     # loading auxilary functions

Dataset:  Features.shape:   # of classes:
vc2c      (310, 6)          2
vc3c      (310, 6)          3
wf24f     (5456, 24)        4
wf4f      (5456, 4)         4
wf2f      (5456, 2)         4
pk        (195, 22)         2


# 2. LSSVM CPU implementation <a class="anchor" id="lssvm_cpu"></a>

In [2]:
import numpy as np
from numpy import dot, exp
from scipy.spatial.distance import cdist

class LSSVM:
    'Class that implements the Least-Squares Support Vector Machine.'
    
    def __init__(self, gamma=1, kernel='rbf', **kernel_params): 
        self.gamma = gamma
        
        self.x        = None
        self.y        = None
        self.y_labels = None
        
        # model params
        self.alpha = None
        self.b     = None
        
        self.kernel = LSSVM.get_kernel(kernel, **kernel_params)
        
           
    @staticmethod
    def get_kernel(name, **params):
        
        def linear(x_i, x_j):                           
            return dot(x_i, x_j.T)
        
        def poly(x_i, x_j, d=params.get('d',3)):        
            return ( dot(x_i, x_j.T) + 1 )**d
        
        def rbf(x_i, x_j, sigma=params.get('sigma',1)):
            if x_i.ndim==x_i.ndim and x_i.ndim==2: # both matrices
                return exp( -cdist(x_i,x_j)**2 / sigma**2 )
            
            else: # both vectors or a vector and a matrix
                return exp( -( dot(x_i,x_i.T) + dot(x_j,x_j.T)- 2*dot(x_i,x_j) ) / sigma**2 )
#             temp = x_i.T - X
#             return exp( -dot(temp.temp) / sigma**2 )
                
        kernels = {'linear': linear, 'poly': poly, 'rbf': rbf}
                
        if kernels.get(name) is None: 
            raise KeyError("Kernel '{}' is not defined, try one in the list: {}.".format(
                name, list(kernels.keys())))
        else: return kernels[name]
        
    
    def opt_params(self, X, y_values):
        sigma = np.multiply( y_values*y_values.T, self.kernel(X,X) )

        A_cross = np.linalg.pinv(np.block([
            [0,                           y_values.T                   ],
            [y_values,   sigma + self.gamma**-1 * np.eye(len(y_values))]
        ]))

        B = np.array([0]+[1]*len(y_values))

        solution = dot(A_cross, B)
        b     = solution[0]
        alpha = solution[1:]
        
        return (b, alpha)
            
    
    def fit(self, X, Y, verboses=0):
        self.x = X
        self.y = Y
        self.y_labels = np.unique(Y, axis=0)
        
        if len(self.y_labels)==2: # binary classification
            # converting to -1/+1
            y_values = np.where(
                (Y == self.y_labels[0]).all(axis=1)
                ,-1,+1)[:,np.newaxis] # making it a column vector
            
            self.b, self.alpha = self.opt_params(X, y_values)
        
        else: # multiclass classification
              # ONE-VS-ALL APPROACH
            n_classes = len(self.y_labels)
            self.b     = np.zeros(n_classes)
            self.alpha = np.zeros((n_classes, len(Y)))
            for i in range(n_classes):
                # converting to +1 for the desired class and -1 for all other classes
                y_values = np.where(
                    (Y == self.y_labels[i]).all(axis=1)
                    ,+1,-1)[:,np.newaxis] # making it a column vector
  
                self.b[i], self.alpha[i] = self.opt_params(X, y_values)

        
    def predict(self, X):
        K = self.kernel(self.x, X)
        
        if len(self.y_labels)==2: # binary classification
            y_values = np.where(
                (self.y == self.y_labels[0]).all(axis=1),
                -1,+1)[:,np.newaxis] # making it a column vector

            Y = np.sign( dot( np.multiply(self.alpha, y_values.flatten()), K ) + self.b)
            
            y_pred_labels = np.where(Y==-1, self.y_labels[0], 
                                     self.y_labels[1])
        
        else: # multiclass classification, ONE-VS-ALL APPROACH
            Y = np.zeros((len(self.y_labels), len(X)))
            for i in range(len(self.y_labels)):
                y_values = np.where(
                    (self.y == self.y_labels[i]).all(axis=1),
                    +1, -1)[:,np.newaxis] # making it a column vector
                Y[i] = dot( np.multiply(self.alpha[i], y_values.flatten()), K ) + self.b[i] # no sign function applied
            
            predictions = np.argmax(Y, axis=0)
            y_pred_labels = np.array([self.y_labels[i] for i in predictions])
            
        return y_pred_labels

Running a single test in all data sets:

In [11]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

for dataset_name in datasets:
    print(dataset_name)

    X = datasets[dataset_name]['features'].values
    Y = datasets[dataset_name]['labels'].values

    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.5)  # Train/Test split
    X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType='min-max') # scaling features
    
    print('linear kernel')
    lssvm = LSSVM(gamma=1, kernel='linear')
    lssvm.fit(X_tr_norm, y_train)
    print('acc_test = ', accuracy_score(dummie_to_multilabel(y_test), 
                         dummie_to_multilabel(lssvm.predict(X_ts_norm))))
    
    print('poly kernel')
    lssvm = LSSVM(gamma=1, kernel='poly', d=2)
    lssvm.fit(X_tr_norm, y_train)
    print('acc_test = ',accuracy_score(dummie_to_multilabel(y_test), 
                         dummie_to_multilabel(lssvm.predict(X_ts_norm))))
    
    print('rbf kernel')
    lssvm = LSSVM(gamma=1, kernel='rbf', sigma=1)
    lssvm.fit(X_tr_norm, y_train)
    print('acc_test = ',accuracy_score(dummie_to_multilabel(y_test), 
                         dummie_to_multilabel(lssvm.predict(X_ts_norm))))
    
    print('\n','#'*100,'\n')

vc2c
linear kernel
acc_test =  0.8258064516129032
poly kernel
acc_test =  0.8387096774193549
rbf kernel
acc_test =  0.8258064516129032

 #################################################################################################### 

vc3c
linear kernel
acc_test =  0.7354838709677419
poly kernel
acc_test =  0.7677419354838709
rbf kernel
acc_test =  0.7870967741935484

 #################################################################################################### 

wf24f
linear kernel
acc_test =  0.6502932551319648
poly kernel
acc_test =  0.8680351906158358
rbf kernel
acc_test =  0.8830645161290323

 #################################################################################################### 

wf4f
linear kernel
acc_test =  0.655791788856305
poly kernel
acc_test =  0.7096774193548387
rbf kernel
acc_test =  0.7291055718475073

 #################################################################################################### 

wf2f
linear kernel
acc_test =  0.6279325

# 3. LSSVM GPU implementation <a class="anchor" id="lssvm_gpu"></a>

This implementation uses `PyTorch`.

In [3]:
import torch

class LSSVM_GPU:
    'Class that implements the Least-Squares Support Vector Machine on GPU.'
    
    def __init__(self, gamma=1, kernel='rbf', **kernel_params): 
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        self.gamma = gamma
        
        self.x        = None
        self.y        = None
        self.y_labels = None
        
        # model params
        self.alpha = None
        self.b     = None
        
        self.kernel = LSSVM_GPU.get_kernel(kernel, **kernel_params) # saving kernel function
        
           
    @staticmethod
    def get_kernel(name, **params):
        
        def linear(x_i, x_j):                           
            return torch.mm(x_i, torch.t(x_j))
        
        def poly(x_i, x_j, d=params.get('d',3)):        
            return ( torch.mm(x_i, torch.t(x_j)) + 1 )**d
        
        def rbf(x_i, x_j, sigma=params.get('sigma',1)):
            if x_i.ndim==x_i.ndim and x_i.ndim==2: # both matrices
                return torch.exp( -torch.cdist(x_i,x_j)**2 / sigma**2 )
            
            else: # both vectors or a vector and a matrix
                return torch.exp( -( torch.dot(x_i,torch.t(x_i)) + torch.dot(x_j,torch.t(x_j))- 2*torch.dot(x_i,x_j) ) 
                                 / sigma**2 )
#             temp = x_i.T - X
#             return exp( -dot(temp.temp) / sigma**2 )
                
        kernels = {'linear': linear, 'poly': poly, 'rbf': rbf}
                
        if kernels.get(name) is None: 
            raise KeyError("Kernel '{}' is not defined, try one in the list: {}.".format(
                name, list(kernels.keys())))
        else: return kernels[name]
        
    
    def opt_params(self, X, y_values):
        sigma = ( torch.mm(y_values, torch.t(y_values)) ) * self.kernel(X,X)

        A_cross = torch.pinverse(torch.cat(( 
            # block matrix
            torch.cat(( torch.tensor(0, dtype=X.dtype, device=self.device).view(1,1),
                        torch.t(y_values)
                      ),dim=1),
            torch.cat(( y_values, 
                        sigma + self.gamma**-1 * torch.eye(len(y_values), dtype=X.dtype, device=self.device) 
                      ),dim=1)
        ),dim=0))

        B = torch.tensor([0]+[1]*len(y_values), dtype=X.dtype, device=self.device).view(-1,1)

        solution = torch.mm(A_cross, B)
        b     = solution[0]
        alpha = solution[1:].view(-1) # 1D array form
        
        return (b, alpha)
            
    
    def fit(self, X, Y, verboses=0):
        # converting to tensors and passing to GPU
        X = torch.from_numpy(X).to(self.device)
        Y = torch.from_numpy(Y).to(self.device)
        self.x = X
        self.y = Y
        self.y_labels = torch.unique(Y, dim=0)
        
        if len(self.y_labels)==2: # binary classification
            # converting to -1/+1
            y_values = torch.where(
                (Y == self.y_labels[0]).all(axis=1)
                ,torch.tensor(-1, dtype=X.dtype, device=self.device)
                ,torch.tensor(+1, dtype=X.dtype, device=self.device)
            ).view(-1,1) # making it a column vector
            
            self.b, self.alpha = self.opt_params(X, y_values)
        
        else: # multiclass classification
              # ONE-VS-ALL APPROACH
            n_classes = len(self.y_labels)
            self.b     = torch.empty(n_classes,         dtype=X.dtype, device=self.device)
            self.alpha = torch.empty(n_classes, len(Y), dtype=X.dtype, device=self.device)
            for i in range(n_classes):
                # converting to +1 for the desired class and -1 for all other classes
                y_values = torch.where(
                    (Y == self.y_labels[i]).all(axis=1)
                    ,torch.tensor(+1, dtype=X.dtype, device=self.device)
                    ,torch.tensor(-1, dtype=X.dtype, device=self.device)
                ).view(-1,1) # making it a column vector
                  
                self.b[i], self.alpha[i] = self.opt_params(X, y_values)

        
    def predict(self, X):
        X = torch.from_numpy(X).to(self.device)
        K = self.kernel(self.x, X)
        
        if len(self.y_labels)==2: # binary classification
            y_values = torch.where(
                (self.y == self.y_labels[0]).all(axis=1)
                ,torch.tensor(-1, dtype=X.dtype, device=self.device)
                ,torch.tensor(+1, dtype=X.dtype, device=self.device)
            )
            
            Y = torch.sign( torch.mm( (self.alpha*y_values).view(1,-1), K ) + self.b)
            
            y_pred_labels = torch.where(Y==-1,        self.y_labels[0],
                                        self.y_labels[1]
                                       ).view(-1) # convert to flat array
        
        else: # multiclass classification, ONE-VS-ALL APPROACH
            Y = torch.empty((len(self.y_labels), len(X)), dtype=X.dtype, device=self.device)
            for i in range(len(self.y_labels)):
                y_values = torch.where(
                    (self.y == self.y_labels[i]).all(axis=1)
                    ,torch.tensor(+1, dtype=X.dtype, device=self.device)
                    ,torch.tensor(-1, dtype=X.dtype, device=self.device)
                )

                Y[i] = torch.mm( (self.alpha[i]*y_values).view(1,-1), K ) + self.b[i] # no sign function applied
            
            predictions = torch.argmax(Y, axis=0)
            y_pred_labels = torch.stack([self.y_labels[i] for i in predictions])
            
        return y_pred_labels

Running a single test in all data sets:

In [13]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train_size = 0.5
for dataset_name in datasets:
    print(dataset_name)

    X = datasets[dataset_name]['features'].values
    Y = datasets[dataset_name]['labels'].values

    X_train, X_test, y_train, y_test = train_test_split(X,Y,train_size=train_size)  # Train/Test split
    X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType='min-max') # scaling features
    
    print('linear kernel')
    lssvm = LSSVM_GPU(gamma=1, kernel='linear')
    lssvm.fit(X_tr_norm, y_train)
    print('acc_test = ', accuracy_score(dummie_to_multilabel(y_test), 
                         dummie_to_multilabel(lssvm.predict(X_ts_norm).cpu().numpy())))
    
    print('poly kernel')
    lssvm = LSSVM_GPU(gamma=1, kernel='poly', d=2)
    lssvm.fit(X_tr_norm, y_train)
    print('acc_test = ',accuracy_score(dummie_to_multilabel(y_test), 
                         dummie_to_multilabel(lssvm.predict(X_ts_norm).cpu().numpy())))
    
    print('rbf kernel')
    lssvm = LSSVM_GPU(gamma=1, kernel='rbf', sigma=1)
    lssvm.fit(X_tr_norm, y_train)
    print('acc_test = ',accuracy_score(dummie_to_multilabel(y_test), 
                         dummie_to_multilabel(lssvm.predict(X_ts_norm).cpu().numpy())))
    
    print('\n','#'*100,'\n')

vc2c
linear kernel
acc_test =  0.8258064516129032
poly kernel
acc_test =  0.8580645161290322
rbf kernel
acc_test =  0.8709677419354839

 #################################################################################################### 

vc3c
linear kernel
acc_test =  0.8580645161290322
poly kernel
acc_test =  0.8387096774193549
rbf kernel
acc_test =  0.8387096774193549

 #################################################################################################### 

wf24f
linear kernel
acc_test =  0.6543255131964809
poly kernel
acc_test =  0.8779325513196481
rbf kernel
acc_test =  0.8947947214076246

 #################################################################################################### 

wf4f
linear kernel
acc_test =  0.6429618768328446
poly kernel
acc_test =  0.7111436950146628
rbf kernel
acc_test =  0.7415689149560117

 #################################################################################################### 

wf2f
linear kernel
acc_test =  0.623900

# 4. Discussing performance <a class="anchor" id="discussing_performance"></a>

The code below was used to evaluate processing time in several data sets and using different kernels:

In [34]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import time
import datetime

small_datasets = ['vc2c', 'vc3c', 'pk']

train_size = 0.5
n_runs = 20

kernels = ['linear', 'poly', 'rbf']

header = ['kernel', 'data set', 'CPU time (mean ± std)', 'GPU time (mean ± std)']
data   = np.empty((len(kernels)*len(datasets), 4), dtype=object)
count=0
for dataset_name in datasets:
    X = datasets[dataset_name]['features'].values
    Y = datasets[dataset_name]['labels'].values

    X_train, X_test, y_train, y_test = train_test_split(X,Y,train_size=train_size)  # Train/Test split
    X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType='min-max') # scaling features

    for kernel in kernels:
        temp_cpu = np.empty(n_runs)
        temp_gpu = np.empty(n_runs)
        for i in range(n_runs):
            lssvm_cpu = LSSVM(gamma=1, kernel=kernel)
            t0 = time.time()
            lssvm_cpu.fit(X_tr_norm, y_train)
            accuracy_score(dummie_to_multilabel(y_test), dummie_to_multilabel(lssvm_cpu.predict(X_ts_norm)))
            t1 = time.time()
            temp_cpu[i] = t1-t0
            
            lssvm_gpu = LSSVM_GPU(gamma=1, kernel=kernel)
            t0 = time.time()
            lssvm_gpu.fit(X_tr_norm, y_train)
            accuracy_score(dummie_to_multilabel(y_test), dummie_to_multilabel(lssvm_gpu.predict(X_ts_norm).cpu().numpy()))
            t1 = time.time()
            temp_gpu[i] = t1-t0
        
        
        data[count] = np.array([kernel, dataset_name, 
                                '{:.2f} ms ± {:.2f} ms'.format(np.mean(temp_cpu)*1e3, np.std(temp_cpu)*1e3), 
                                '{:.2f} ms ± {:.2f} ms'.format(np.mean(temp_gpu)*1e3, np.std(temp_gpu)*1e3)])
        count+=1
        print("Done {}/{} at {}.".format(count, len(data), datetime.datetime.now()))
        
df = pd.DataFrame(data, columns=header)

Done 1/18 at 2019-08-13 21:53:24.580535.
Done 2/18 at 2019-08-13 21:53:25.403790.
Done 3/18 at 2019-08-13 21:53:27.114317.
Done 4/18 at 2019-08-13 21:53:30.465400.
Done 5/18 at 2019-08-13 21:53:33.482345.
Done 6/18 at 2019-08-13 21:53:36.818694.
Done 7/18 at 2019-08-13 22:20:52.139768.
Done 8/18 at 2019-08-13 22:54:52.164808.
Done 9/18 at 2019-08-13 23:29:40.950704.
Done 10/18 at 2019-08-13 23:58:46.899122.
Done 11/18 at 2019-08-14 00:26:35.182333.
Done 12/18 at 2019-08-14 00:53:53.112603.
Done 13/18 at 2019-08-14 01:20:36.839217.
Done 14/18 at 2019-08-14 01:48:04.472977.
Done 15/18 at 2019-08-14 02:15:24.448660.
Done 16/18 at 2019-08-14 02:15:25.287985.
Done 17/18 at 2019-08-14 02:15:25.730238.
Done 18/18 at 2019-08-14 02:15:26.173697.
CPU times: user 19h 18s, sys: 2h 33min 54s, total: 21h 34min 12s
Wall time: 4h 22min 2s


In [41]:
# saving results
# filename = "df.csv"
# df.to_csv(filename, sep='\t', index=False)

# loading results
df = pd.read_csv("df.csv", sep='\t')
df.sort_values(by=['data set', 'kernel'])

,kernel,data set,CPU time (mean ± std),GPU time (mean ± std)
15,linear,pk,7.02 ms ± 1.29 ms,34.82 ms ± 23.42 ms
16,poly,pk,11.56 ms ± 0.53 ms,10.45 ms ± 0.25 ms
17,rbf,pk,11.05 ms ± 0.71 ms,11.01 ms ± 0.14 ms
0,linear,vc2c,14.30 ms ± 4.36 ms,22.71 ms ± 21.36 ms
1,poly,vc2c,23.18 ms ± 7.49 ms,17.88 ms ± 3.64 ms
2,rbf,vc2c,33.48 ms ± 18.59 ms,51.93 ms ± 41.71 ms
3,linear,vc3c,53.19 ms ± 22.06 ms,114.18 ms ± 25.69 ms
4,poly,vc3c,52.88 ms ± 14.15 ms,97.86 ms ± 28.40 ms
5,rbf,vc3c,57.54 ms ± 14.44 ms,109.17 ms ± 34.44 ms
6,linear,wf24f,42767.05 ms ± 3093.38 ms,38998.68 ms ± 204.05 ms


As expected the performance on the GPU was better when the data set was big (*wf24*, *wf4f*, *wf2f*), on the small data sets the CPU version was usually faster and sometimes equivalent.